In [ ]:
%pylab notebook

## Exercise 5: Simple Hay et al. model neuron simulation with one synaptic input

In this exercise you will work with a 'state-of-the-art' multi-compartmental model form literature, namely the Hay model (Hay et al. (2011) PLoS Comput Biol 7: e1002107, doi:10.1371/journal.pcbi.1002107). This model has an anatomically reconstructed dendritic morphology (Figure 1), and many active conductances that have been added and fitted to experimental data. Many such cell models are now freely available from online data bases like ModelDB (http://senselab.med.yale.edu/modeldb/), and if you want to make your own model, dendritic morphologies can be downloaded from neuromorpho.org. Scientific papers based solely on simulations with such cell models taken from lit-
erature and data bases are not uncommon. In this exercise you will be given a simple example script, exercise 5.py,
based on LFPy and Neuron, that you can adapt to solve this exercise.

<img src="Hay-neuron.png" width = "100">

Figure 1: Reconstructed morphology of a rat neocortical pyramidal cell by Hay et al. (2011). Experimental recording sites are indicated by schematic electrodes at the soma (black), proximal apical dendrite (400 $\mu$m from the soma, blue) and distal apical site (620 $\mu$m from the soma, red). Taken from Figure 1 in Hay et al. (2011),
http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1002107

__(i)__ Make the cell spike by giving it one or more synaptic inputs. Experiment with different synapse numbers, locations and synaptic strengths. Have fun! 

__(ii)__ Plot the 'raw' and normalized subthreshold somatic voltage response from synaptic inputs for increasing distances between the synaptic input and the soma. What do you see?

__(iii)__ Plot the somatic membrane potential response to a spike train input to a synapse in the distal apical dendrite when the neuron is passive, that is, has no active conductances. The incoming spike train should contain three spikes, with an even interspike interval of 40 ms. How do the synaptic input sum at the input site? How do they sum in the soma compartment? How does it change if the cell has all the original active conductances? (Note that the documentation in the code explains how to turn active conductances on and off.)

__(iv)__ [Optional] In the release of the LFPy program (Linden et al. (2014). Frontiers in Neuroinformatics 7:41 (2014), doi: 10.3389/fninf.2013.00041), several example applications were included. They are available through the LFPy website at http://lfpy.github.io/. Test out some of them (but note that the example scripts also produce plots of extracellular potentials which we will go through later in the course).

In [1]:
import os
retval = os.getcwd()
print("Current working directory %s" % retval)
os.chdir("hay_model/mod/")
!nrnivmodl
os.chdir(retval)

from os.path import join
import numpy as np
import neuron
import LFPy
from hay_model.hay_active_declarations import active_declarations
nrn = neuron.h

Current working directory /Users/geih/Documents/Undervisning/Fys388_2019/Exercises2019_online/Exercise05
/Users/geih/Documents/Undervisning/Fys388_2019/Exercises2019_online/Exercise05/hay_model/mod
CaDynamics_E2.mod Ca_HVA.mod Ca_LVAst.mod Ih.mod Im.mod K_Pst.mod K_Tst.mod NaTa_t.mod NaTs2_t.mod Nap_Et2.mod SK_E2.mod SKv3_1.mod
CaDynamics_E2.mod Ca_HVA.mod Ca_LVAst.mod Ih.mod Im.mod K_Pst.mod K_Tst.mod NaTa_t.mod NaTs2_t.mod Nap_Et2.mod SK_E2.mod SKv3_1.mod
"/Applications/NEURON-7.7/nrn/share/nrn/libtool" --tag=CC --mode=compile gcc -DHAVE_CONFIG_H  -I. -I.. -I"/Applications/NEURON-7.7/nrn/include/nrn" -I"/Applications/NEURON-7.7/nrn/x86_64/lib"      -g -O2 -c -o mod_func.lo mod_func.c
libtool: compile:  gcc -DHAVE_CONFIG_H -I. -I.. -I/Applications/NEURON-7.7/nrn/include/nrn -I/Applications/NEURON-7.7/nrn/x86_64/lib -g -O2 -c mod_func.c  -fno-common -DPIC -o .libs/mod_func.o
"/Applications/NEURON-7.7/nrn/share/nrn/libtool" --tag=CC --mode=link gcc -module  -g -O2    -headerpad_max_inst

In [ ]:
def return_cell(synaptic_y_pos=900, conductance_type='active', weight=0.001, input_spike_train=np.array([10.])):
    """
    Runs a NEURON simulation and returns an LFPy cell object for a single synaptic input.
    :param synaptic_y_pos: position along the apical dendrite where the synapse is inserted.
    :param conductance_type: Either 'active' or 'passive'. If 'active' all original ion-channels are included,
           if 'passive' they are all removed, yielding a passive cell model.
    :param weight: Strength of synaptic input.
    :param input_spike_train: Numpy array containing synaptic spike times
    :return: cell object where cell.imem gives transmembrane currents, cell.vmem gives membrane potentials.
             See LFPy documentation for more details and examples.
    """
    nrn('forall delete_section()')
    model_path = join('hay_model')
    neuron.load_mechanisms(join(model_path, 'mod'))
    cell_parameters = {
        'morphology': join(model_path, 'cell1.hoc'),
        'v_init': -65,
        'passive': False,
        'nsegs_method': 'lambda_f',
        'lambda_f': 100,
        'dt': 2**-3,  # Should be a power of 2
        'tstart': -200,
        'tstop': 200,
        'custom_code': [join(model_path, 'custom_codes.hoc')],
        'custom_fun': [active_declarations],
        'custom_fun_args': [{'conductance_type': conductance_type}],
    }
    cell = LFPy.Cell(**cell_parameters)
    synapse_parameters = {
        'idx': cell.get_closest_idx(x=0., y=synaptic_y_pos, z=0.),
        'e': 0.,
        'syntype': 'ExpSyn',
        'tau': 10.,
        'weight': weight,
        'record_current': True,
    }
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(input_spike_train)
    cell.simulate(rec_imem=True, rec_vmem=True)
    return cell

In [ ]:
def plot_cell(cell):
    cell_plot_idx = 0
    subplots_adjust(hspace=0.5)  # Adjusts the vertical distance between panels.
    subplot(121, aspect='equal', xticks=[], xlabel='x', ylabel='y [$\mu m$]')
    [plot([cell.xstart[idx], cell.xend[idx]], [cell.ystart[idx], cell.yend[idx]], c='k') 
     for idx in range(cell.totnsegs)]
    plot(cell.xmid[cell.synidx], cell.ymid[cell.synidx], 'ro')
    subplot(222, title='Membrane potential', xlabel='Time [ms]', ylabel='mV')
    plot(cell.tvec, cell.vmem[cell_plot_idx, :])
    subplot(224, title='Transmembrane currents', xlabel='Time [ms]', ylabel='nA')
    plot(cell.tvec, cell.imem[cell_plot_idx, :])
    show()

In [ ]:
cell = return_cell()
plot_cell(cell)